In [47]:
! pip install mysql-connector-python

In [48]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, KFold
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import matplotlib
from matplotlib import pyplot as plt
from nltk.classify import NaiveBayesClassifier
from nltk.classify.util import accuracy as nltk_accuracy

import mysql.connector
from mysql.connector import Error
# import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [49]:
connection = mysql.connector.connect(
        host='ds50-mysql-do-user-9644544-0.b.db.ondigitalocean.com',
        database='ds50',
        user='ds50',
        password='AVNS_4ybSd0CoPKnCL5F',
        port = '25060')
if connection.is_connected():
        data = pd.read_sql( '''SELECT * FROM REVIEW LIMIT 5000 
        ''', connection)
        connection.close()
data=pd.DataFrame(data)


In [50]:
data.head(50)


In [51]:
data.info()

In [84]:
data = data[(data['review_text'].str.len() > 3) & (data['review_text'].str.len() < 3000)]


In [85]:
data.shape

In [87]:
def isfloat(num):
    try:
        float(num)
        return True
    except ValueError:
        return False


In [90]:
import re  
import nltk
 
nltk.download('stopwords')
 
# to remove stopword
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

# for Stemming propose
from nltk.stem.porter import PorterStemmer
corpus = []

def remove_punctuation(text):
    
    import string
     
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)
for i in range(0, len(data)):
    if data['review_text'].iloc[i].isdigit() or isfloat(data['review_text'].iloc[i]) :
        if float(data['review_text'].iloc[i])>=0 and float(data['review_text'].iloc[i])<= 1:
            data['review_text'].iloc[i]="bad"
        elif float(data['review_text'].iloc[i])>1 and float(data['review_text'].iloc[i])<= 2:
            data['review_text'].iloc[i]="not bad"
        elif float(data['review_text'].iloc[i])>2 and float(data['review_text'].iloc[i])<= 3:
            data['review_text'].iloc[i]="average"
        elif float(data['review_text'].iloc[i])>3 and float(data['review_text'].iloc[i])<= 4:
            data['review_text'].iloc[i]="good"
        elif float(data['review_text'].iloc[i])>4 and float(data['review_text'].iloc[i])<= 5:
            data['review_text'].iloc[i]="very good"
    else : 
        review = remove_punctuation(data['review_text'].iloc[i])

        # convert all cases to lower cases
        review = review.lower()

        # split to array(default delimiter is " ")
        review = review.split()

        # creating PorterStemmer object to
        # take main stem of each word
        ps = PorterStemmer()

        # loop for stemming each word
        # in string array at ith row   
        review = [ps.stem(word) for word in review
                    if not word in set(stopwords.words('english'))]

        # rejoin all string array elements
        # to create back into a string
        review = ' '.join(review) 

        # append each string to create
        # array of clean text
        corpus.append(review)
        data['review_text'].iloc[i]=review

****Translate****

In [91]:
!pip install -U deep-translator

In [92]:
!pip install langdetect

In [97]:
import deep_translator as dt




for i in range(1,1000):
    data.loc[i,'review_text'] = dt.GoogleTranslator(source='auto', target='en').translate(data.loc[i,'review_text'])
        


In [69]:
from langdetect import detect 

from deep_translator import GoogleTranslator

for i in range(0, len(data)):                            

    texte=data['review_text'].iloc[i]

    if detect(texte)!="en"and texte.isalpha():

        translated = GoogleTranslator(source='auto', target='en').translate(text=texte)

        data['review_text'].iloc[i]== translated

data.head()

**Ratings count**

In [98]:
import seaborn as sns
sns.countplot(x=data['rating'],data=data)
plt.show()

In [118]:
ratings_equal_zero=data[data['rating']==0]


In [102]:
data=data[['review_id', 'rating', 'review_text']]
data.head()

In [103]:
rate_id_one = []
rate_id_two = []
rate_id_three = []
rate_id_four = []
rate_id_five =[]


for review_text, rating in zip(data['review_text'], data['rating']):
    line = review_text.split(" ")
    dic = {}
    if rating == 1:
        for word in line:
            dic[word] = True
        ireru = (dic, 1)
        rate_id_one.append(ireru)
    elif rating == 2:
        for word in line:
            dic[word] = True
        ireru = (dic, 2)
        rate_id_two.append(ireru) 
    elif rating == 3:
        for word in line:
            dic[word] = True
        ireru = (dic, 3)
        rate_id_three.append(ireru)
    elif rating == 4:
        for word in line:
            dic[word] = True
        ireru = (dic, 4)
        rate_id_four.append(ireru)
    else:
        for word in line:
            dic[word] = True
        ireru = (dic, 5)
        rate_id_five.append(ireru)


In [104]:
threshold = 0.8
num_one = int(threshold * len(rate_id_one))
num_two = int(threshold * len(rate_id_two))
num_three = int(threshold * len(rate_id_three))
num_four = int(threshold * len(rate_id_four))
num_five = int(threshold * len(rate_id_five))

features_train = rate_id_one[:num_one] + rate_id_two[:num_two] + rate_id_three[:num_three] + rate_id_four[:num_four] + rate_id_five[:num_five]
features_test = rate_id_one[num_one:] + rate_id_two[num_two:] + rate_id_three[num_three:] + rate_id_four[num_four:] + rate_id_five[num_five:]
print("Number of training datapoints:", len(features_train))
print("Number of test datapoints:", len(features_test))

In [105]:
classifier = NaiveBayesClassifier.train(features_train)
print('Accuracy of the classifier:', nltk_accuracy(classifier, features_test))

The cause may be that there is a wide range of choices from 1 to 5 and that the absolute number of low-rated sentence data is insufficient. For example, if you choose between 1 and 5 rates, it may improve.

Alternatively, if other methods are used, the accuracy may be further improved, so this will be an issue for the future.

words influenced the predicted numbers during the training.

In [106]:
N = 30
print('Top ' + str(N) + ' most informative words:')
for i, item in enumerate(classifier.most_informative_features()[:N]):
    print(str(i+1) + '. ' + item[0]) 

In [107]:
def extract_features(words):
    return dict([(word, True) for word in words])
#Divide into words like the large amount of review text earlier


print("Clothes review predictions:")

 
for i in range(0, len(data)):
    if data['rating'].iloc[i]==0 :
        input_review = data['review_text'].iloc[i]
        features = extract_features(input_review.split())
        probabilities = classifier.prob_classify(features)
        predicted_sentiment = probabilities.max()
        data['rating'].iloc[i]=predicted_sentiment
        

In [108]:
ratings_equal_zero = data[data['rating']==0] 
ratings_equal_zero.head()

there is no ratings equal to 0

In [109]:
data.head(30)